<a href="https://colab.research.google.com/github/vines2021/Image-caption-Generator/blob/main/Image_caption_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download("punkt")



# Define parameters
embedding_size = 300
vocab_size = 10000
max_length = 40
image_shape = (299, 299, 3)

# Load InceptionV3 model
image_model = InceptionV3(include_top=False, weights='imagenet')

# Remove the last layer and add a new one
new_input = Input(shape=image_shape)
hidden_layer = image_model(new_input)
hidden_layer = Dropout(0.5)(hidden_layer)
hidden_layer = Dense(embedding_size, activation='relu')(hidden_layer)

# Define the image model
image_model = Model(inputs=new_input, outputs=hidden_layer)

# Define the caption model
input_layer = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_size)(input_layer)
hidden_layer = LSTM(256)(embedding_layer)
output_layer = Dense(vocab_size, activation='softmax')(hidden_layer)

# Compile the caption model
caption_model = Model(inputs=input_layer, outputs=output_layer)
caption_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Define a function to generate captions for images
def generate_caption(image_path):
  # Load and preprocess the image
  img = image.load_img(image_path, target_size=(image_shape[0], image_shape[1]))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  
  # Get the image features
  image_features = image_model.predict(img)
  
  # Initialize the caption sequence
  caption_input = np.zeros((1, max_length))
  
  # Initialize the caption
  caption = []
  
  # Loop through the sequence and generate the next word
  for i in range(max_length):
    # Predict the next word
    predicted_word = caption_model.predict(caption_input)
    
    # Get the index of the most likely word
    predicted_word_index = np.argmax(predicted_word[0])
  #  tokenizer= word_tokenize()
    
    # Convert the index to the corresponding word
    predicted_word = tokenizer.index_word[predicted_word_index]
    
    # Append the word to the caption
    caption.append(predicted_word)
    
    # Check for the end of the caption
    if predicted_word == '<end>':
      break
      
    # Update the input sequence
    caption_input[0][i+1] = predicted_word_index
  
  # Join the words to form the final caption
  final_caption = ' '.join(caption[1:-1])
  
  return final_caption
  
# Test the model on an image
image_path = ("/content/WhatsApp Image 2023-04-24 at 16.53.55.jpg")
caption = generate_caption(image_path)
print(caption)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1/1 [==============================] - 0s 457ms/step


NameError: ignored